In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import os
import pandas as pd

from dynamicslib.consts import muEM
from dynamicslib.common import manifold_stepoffs, prop_multiple, jacobi_constant, coupled_stm_eom, eom
from dynamicslib.integrator import dop853
from dynamicslib.common_targetters import *
from dynamicslib.targeter import dc_square
from numba import njit

from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

fam1 = "L1 Halo"
fam2 = "L2 Halo"
JC_target = 3.1

root = os.getcwd()
df1 = pd.read_csv(f"{root}/database/{fam1}.csv").set_index("Index")
df2 = pd.read_csv(f"{root}/database/{fam2}.csv").set_index("Index")
ind1 = np.argmin(np.abs(df1["Jacobi Constant"]-JC_target))
ind2 = np.argmin(np.abs(df2["Jacobi Constant"]-JC_target))
stab1 = df1.iloc[ind1]["Stability Index"]
stab2 = df2.iloc[ind2]["Stability Index"]

In [ ]:
int_tol = 1e-12
targetter = JC_fixed_spatial_perpendicular(int_tol, JC_target, muEM)

In [ ]:
X1 = df1[["Initial x", "Initial z", "Initial vy", "Period"]].values[ind1]
X2 = df2[["Initial x", "Initial z", "Initial vy", "Period"]].values[ind2]
X1[-1] /= 2
X2[-1] /= 2

x02 = targetter.get_x0(X2)

In [ ]:
X1, _, _ = dc_square(X1, targetter.f_df_stm, 1e-11, debug=True)
x01, tf1 = targetter.get_x0(X1), targetter.get_tf(X1)
X2, _, _ = dc_square(X2, targetter.f_df_stm, 1e-11, debug=True)
X2[1] *= -1
x02, tf2 = targetter.get_x0(X2), targetter.get_tf(X2)

In [ ]:
@njit(cache=True)
def moon(t, x, mu):
    return x[0] - (1 - mu)


@njit(cache=True)
def term(t, x, mu):
    return x[2] if x[0] > 1. else np.nan


@njit(cache=True)
def exit_right(t, x, mu):
    return x[0] - 1.6


@njit(cache=True)
def exit_left(t, x, mu):
    return x[0] - 0.7


n_event = 2

term.terminal = n_event
term.direction = 0
moon.terminal = n_event
moon.direction = 0
exit_right.terminal = 1
exit_left.terminal = 1
exit_right.direction = 1
exit_left.direction = -1

events = [exit_right, exit_left, moon]

In [ ]:
_, x0u_1 = manifold_stepoffs(x01, tf1, 50, 1e-3, muEM, 1e-12)
x0s_2, _ = manifold_stepoffs(x02, tf2, 50, 1e-3, muEM, 1e-12)

In [ ]:
out1 = solve_ivp(eom, (0.0, tf1), x01, "DOP853", atol=1e-12, rtol=1e-12, args=(muEM,))
out2 = solve_ivp(eom, (0.0, -tf2), x02, "DOP853", atol=1e-12, rtol=1e-12, args=(muEM,))

In [ ]:
manifolds1 = prop_multiple(np.array(x0u_1), events, 10.0, muEM, 1e-10)
print("done w 1")
manifolds2 = prop_multiple(np.array(x0s_2), events, -10.0, muEM, 1e-10)
print("done w 2")

In [ ]:
ie = n_event - 1
ye1 = {
    k: v.y_events[-1][ie]
    for k, v in manifolds1.items()
    if len(v.y_events[-1]) == n_event
}
ye2 = {
    k: v.y_events[-1][ie]
    for k, v in manifolds2.items()
    if len(v.y_events[-1]) == n_event
}
ye1 = {k: np.array([*v[:3], *v[3:] / 10]) for k, v in ye1.items()}
ye2 = {k: np.array([*v[:3], *v[3:] / 10]) for k, v in ye2.items()}

dists = [
    ((k1, k2), np.linalg.norm(v1 - v2))
    for k2, v2 in ye2.items()
    for k1, v1 in ye1.items()
]
best_set = dists[np.argmin([d[1] for d in dists])]
i1, i2 = best_set[0]

In [ ]:
%matplotlib ipympl
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(projection="3d")
for dictionary, color in zip([manifolds1, manifolds2], ["r", "b"]):
    for ind in dictionary.keys():
        out = dictionary[ind]
        x, y, z = out.y[:3].copy()
        ax.plot(x, y, z, f"-{color}", alpha=0.025)
ax.plot(out1.y[0], out1.y[1], out1.y[2], "-k")
ax.plot(out2.y[0], out2.y[1], out2.y[2], "-k")
ax.plot(*manifolds1[i1].y[:3], f"-r", lw=2)
ax.plot(*manifolds2[i2].y[:3], f"-b", lw=2)
ax.set(
    xlim=(0.75, 1.25),
    ylim=(-0.2, 0.2),
    zlim=(-0.2, 0.2),
    xlabel="x [LU]",
    ylabel="y [LU]",
    aspect='equal'
)
# ax.axis("equal")
plt.grid(True)
plt.show()